# Topic Modeling on Bukowski's Poems

On the first post about Bukowski's poems we explored the top words and their polarity. From inspection these groups seemed to be associated to 4 main topics, which also happen to be mentioned on the writter's legacy website. It would be interesting to see if these same topics show up when applicating a generative statistical modeling, such as the Latent Dirichlet Allocation (LDA). To do this and visualize the results I'll use the pyLDAvis and scikit-learn packages.

In [2]:
## Setup and importing libraries.
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')

from tqdm import tqdm
import pandas as pd
import gensim
import pyLDAvis
import pyLDAvis.sklearn

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

pyLDAvis.enable_notebook()

In [3]:
## Load the pickle file.
content = pd.read_pickle('data/content.pkl')

## Add each poem as an element of a list.
raw_poems = []
for p in content.poem.map(int).unique():
    flat_poem = ' '.join([text for text in content.loc[content.poem == p, 'text']])
    raw_poems.append(flat_poem)

In order to perform the LDA, we need to generate a document-term matrix representation of all the columns. Rows in our matrix represent words, while columns correspond to different words. The `CountVectorizer` function helps us do this, and it also performs some pre-processing (removing stop-words, generating bigrams, setting minimum word frequencies, setting everything to lowercase).

In [31]:
## Generate sparse document-term matrix.
dtm_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                ngram_range = (1,1), 
                                min_df = 10,
                                max_df = 0.2)
dtm_poems = dtm_vectorizer.fit_transform(raw_poems)
print dtm_poems.shape

(1363, 1883)


## Fitting and Visualizing the LDA Model
To fit the LDA algorithm, we are required to set the number of topics it will associate the words with. This is an arbitrary decision, and in this case I'll start with 7.

In [32]:
lda_model = LatentDirichletAllocation(n_topics=7)
lda_model.fit(dtm_poems)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=7, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [33]:
pyLDAvis.sklearn.prepare(lda_model, dtm_poems, dtm_vectorizer, mds='tsne')

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
1      29.255093        1       1  -19.757645   80.262382
6      27.745719        1       2   54.838648  -13.032907
4      19.686873        1       3   64.774753 -143.175641
5       8.248519        1       4 -133.664547   12.927086
3       7.531803        1       5  177.104671  -47.139952
0       5.218277        1       6  -54.990799  -95.626408
2       2.313716        1       7 -109.080534  174.093650, topic_info=     Category        Freq       Term       Total  loglift  logprob
term                                                              
968   Default  363.000000       love  363.000000  30.0000  30.0000
559   Default  197.000000       feel  197.000000  29.0000  29.0000
1764  Default  374.000000     walked  374.000000  28.0000  28.0000
248   Default  353.000000       came  353.000000  27.0000  27.0000
1802  Default  391.000000       went  391.000000  26.0000  26.0000
1206  Default  167.000000      poems  167.000000  25.0000  25.0000
1294  Default  227.000000       read  227.000000  24.0000  24.0000
1681  Default  342.000000       told  342.000000  23.0000  23.0000
253   Default  331.000000        car  331.000000  22.0000  22.0000
1863  Default  168.000000      write  168.000000  21.0000  21.0000
94    Default  140.000000        bar  140.000000  20.0000  20.0000
421   Default  365.000000       didn  365.000000  19.0000  19.0000
671   Default  250.000000        god  250.000000  18.0000  18.0000
57    Default  175.000000        ask  175.000000  17.0000  17.0000
1371  Default  244.000000        sat  244.000000  16.0000  16.0000
1867  Default  112.000000    writing  112.000000  15.0000  15.0000
1205  Default  105.000000       poem  105.000000  14.0000  14.0000
956   Default  247.000000     looked  247.000000  13.0000  13.0000
439   Default  101.000000       dogs  101.000000  12.0000  12.0000
58    Default  258.000000      asked  258.000000  11.0000  11.0000
693   Default  162.000000      great  162.000000  10.0000  10.0000
1864  Default   98.000000     writer   98.000000   9.0000   9.0000
1577  Default  138.000000      stood  138.000000   8.0000   8.0000
438   Default  118.000000        dog  118.000000   7.0000   7.0000
422   Default  151.000000        die  151.000000   6.0000   6.0000
117   Default  157.000000      began  157.000000   5.0000   5.0000
1209  Default   84.000000      poets   84.000000   4.0000   4.0000
1005  Default  179.000000      maybe  179.000000   3.0000   3.0000
792   Default  122.000000        hot  122.000000   2.0000   2.0000
1375  Default  187.000000        saw  187.000000   1.0000   1.0000
...       ...         ...        ...         ...      ...      ...
740    Topic7   26.556239       hate   62.885233   2.9043  -4.2979
834    Topic7    5.604808   japanese   13.408474   2.8941  -5.8535
1848   Topic7    6.231341    workers   14.937310   2.8920  -5.7475
323    Topic7    6.918893   clothing   17.175326   2.8571  -5.6429
646    Topic7    4.632497     gather   11.533651   2.8541  -6.0440
312    Topic7    9.152006     clever   22.896403   2.8493  -5.3632
1608   Topic7   12.614296   sunlight   31.740548   2.8436  -5.0423
627    Topic7    6.086900     friday   15.439215   2.8355  -5.7710
340    Topic7   11.613171   computer   29.493776   2.8343  -5.1250
1539   Topic7    5.388438      split   13.692902   2.8337  -5.8929
1159   Topic7    4.908893      pause   12.646264   2.8200  -5.9861
559    Topic7   61.463276       feel  197.437699   2.5993  -3.4587
426    Topic7   13.737677  different   39.823723   2.7020  -4.9570
1205   Topic7   29.413628       poem  105.296331   2.4910  -4.1957
613    Topic7   15.201470    forever   48.647507   2.6031  -4.8557
305    Topic7   21.478257       city   87.767831   2.3587  -4.5101
604    Topic7   11.069236     flying   34.903904   2.6179  -5.1730
1267   Topic7    7.725054    quickly   21.130503   2.7601  -5.5327
968    Topic7   42

Examining each of the word clusters, or topics, we see some very general ones and others having too much noise. However there are a couple of interesting ones:

- **Cluster 3:** General topic seems to be around love and dead. Other related words are lives, laughter, reason and madness.
- **Cluster 5:** Horse tracks. The author's visits to this place are also a common subject in his writtings, and this gets reflected here.
- **Cluster 6:** Alcohol and its multiple manifestations.
- **Cluster 7:** Music and art, which are also commonly present on Bukowski's life.

## Expanding Number of Topics
If we define a larger number of clusters for the LDA model, these will tend to become more granular, particularly for the smaller ones. Below I redo the exercise but for 25 groups, I won't go into details but the user is encouraged to explore. 

In [341]:
lda_model = LatentDirichletAllocation(n_topics=25)
lda_model.fit(dtm_poems)
pyLDAvis.sklearn.prepare(lda_model, dtm_poems, dtm_vectorizer, R=20)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
13     24.664318        1       1 -0.074051  0.053798
11     23.229509        1       2 -0.064008  0.164120
23     13.975983        1       3 -0.110824  0.023524
17      5.994711        1       4 -0.103482  0.039258
19      4.588245        1       5 -0.050417  0.175548
16      3.967552        1       6 -0.095725  0.146836
12      3.691477        1       7 -0.157568  0.036788
7       3.474120        1       8 -0.071607 -0.058344
0       3.381880        1       9 -0.116777 -0.096513
24      2.534305        1      10 -0.113686 -0.139155
6       2.319543        1      11 -0.154014 -0.115928
15      2.103734        1      12 -0.057420  0.112983
2       1.034710        1      13  0.014564 -0.159526
10      0.838749        1      14 -0.091621 -0.140314
9       0.741720        1      15  0.077131  0.084599
4       0.718235        1      16  0.015501 -0.144745
5       0.493983        1      17  0.079507 -0.032183
8       0.460002        1      18  0.215517 -0.068103
20      0.369913        1      19  0.050535  0.046696
1       0.341854        1      20  0.105145 -0.079326
18      0.294118        1      21  0.200692  0.020006
14      0.281115        1      22  0.148753  0.015397
22      0.277816        1      23  0.195967  0.009862
3       0.157482        1      24  0.106849  0.036665
21      0.064927        1      25  0.051041  0.068056, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
499   Default  1551.000000          like  1551.000000  15.0000  15.0000
549   Default   670.000000           man   670.000000  14.0000  14.0000
745   Default  1174.000000          said  1174.000000  13.0000  13.0000
646   Default   426.000000        people   426.000000  12.0000  12.0000
26    Default   202.000000           ask   202.000000  11.0000  11.0000
608   Default   624.000000           old   624.000000  10.0000  10.0000
465   Default   606.000000          know   606.000000   9.0000   9.0000
122   Default   352.000000           car   352.000000   8.0000   8.0000
346   Default   256.000000           god   256.000000   7.0000   7.0000
195   Default   321.000000          dead   321.000000   6.0000   6.0000
582   Default   149.000000         music   149.000000   5.0000   5.0000
701   Default   154.000000         radio   154.000000   4.0000   4.0000
520   Default   375.000000          look   375.000000   3.0000   3.0000
233   Default   355.000000         drink   355.000000   2.0000   2.0000
219   Default   669.000000           don   669.000000   1.0000   1.0000
243    Topic1    38.831306           dry    40.002726   1.3701  -6.2522
630    Topic1    23.788732       panties    24.848793   1.3562  -6.7423
79     Topic1    22.848634         board    23.940233   1.3531  -6.7826
1027   Topic1    23.679108     years ago    24.835435   1.3521  -6.7469
619    Topic1    18.604931      overhead    19.750286   1.3401  -6.9880
316    Topic1    35.798173          form    38.173050   1.3356  -6.3336
835    Topic1    34.051147      stairway    37.009811   1.3165  -6.3836
487    Topic1    46.603103           leg    51.339684   1.3030  -6.0698
640    Topic1    26.236380         party    29.192068   1.2931  -6.6443
806    Topic1    29.581717          slow    33.316057   1.2809  -6.5243
668    Topic1    21.157730        plenty    23.865210   1.2794  -6.8595
423    Topic1    78.174175         horse    90.450106   1.2540  -5.5525
1011   Topic1    49.607797         worst    57.652550   1.2495  -6.0073
343    Topic1    36.405722          glad    42.920500   1.2352  -6.3167
320    Topic1    37.886256          free    44.843289   1.2312  -6.2769
...       ...          ...           ...          ...      ...      ...
694   Topic25     0.051320       quickly    20.998890   1.3255  -6.9413
716   Topic25     0.051269  refrigerator    20.9956